# ※ <한강홍수통제소_OPEN API를 이용한 수위 데이터 수집> 따라하기
- 한강홍수통제소 OPEN API
> http://www.hrfco.go.kr/web/openapiPage/openApi.do

In [1]:
SERVICE_KEY = "E49E53F1-C36B-4EB5-B8F6-F0C91AC036F0"

In [2]:
import pandas as pd
import numpy as np
import requests
import calendar
import time

In [3]:
# 수위 관측소 제원 요청
url = f'http://api.hrfco.go.kr/{SERVICE_KEY}/waterlevel/info.json'
response = requests.get(url)

data = response.json()

In [4]:
# 수위 관측소 필터링
for con in data['content']:
    if '서울' in con['obsnm']:
        print(con['obsnm'], con['wlobscd'])
    if '팔당'  in con['obsnm']:
        print(con['obsnm'], con['wlobscd'])

팔당댐 1017678
남양주시(팔당댐) 1017690
남양주시(팔당대교) 1018610
팔당대교 1018611
서울시(광진교) 1018640
서울시(대곡교) 1018655
서울시(대치교) 1018658
서울시(청담대교) 1018662
서울시(창동교) 1018669
서울시(월계2교) 1018670
서울시(중랑교) 1018675
서울시(잠수교) 1018680
서울시(한강대교) 1018683
서울시(너부대교) 1018695
서울시(오금교) 1018697
서울시(행주대교) 1019630


In [5]:
bridge = {
    "남양주시(팔당대교)": 1018610,
    "서울시(광진교)": 1018640,
}

In [9]:
# bridge 변수에 저장한 다리의 수위 자료 요청
for name, code in bridge.items():
    for year in range(2012, 2023):
        ms, me = (5, 11) if year != 2022 else (5, 8)
        for month in range(ms, me):
            weekday, end = calendar.monthrange(year, month)
            sdate = f"{year}{month:02}010000" 
            edate = f"{year}{month:02}{end:02}2350"
            url = f"http://api.hrfco.go.kr/{SERVICE_KEY}/waterlevel/list/10M/{code}/{sdate}/{edate}.json"            
            response = requests.get(url)
            df = pd.DataFrame(response.json()['content'])
            df.to_csv(f"C:/JupyterProject/5. 한강_수위예측_AI경진대회_DACON_(22.07.25.-09.23.)/한강홍수통제소/{year}{month:02}_{name}.csv")
            time.sleep(3)

# ※ csv 파일 불러오기
- [파이썬 기초] 특정 폴더에 있는 파일명 추출 하는법
> https://bit.ly/3KkYwfp
- [파이썬] Python에서 특정 문자열 포함여부 확인하는 방법(실무에서 많이 사용합니다.)
> https://bit.ly/3AM4BhW

In [10]:
from glob import glob
import os
import warnings        
warnings.filterwarnings("ignore") 

In [11]:
# '한강홍수통제소' 라는 폴더명을 가진 디렉토리로 이동
os.chdir("./한강홍수통제소")

# 해당 폴더 내에 어떤 파일들이 있는지 확인
print(os.listdir())

['201205_남양주시(팔당대교).csv', '201205_서울시(광진교).csv', '201206_남양주시(팔당대교).csv', '201206_서울시(광진교).csv', '201207_남양주시(팔당대교).csv', '201207_서울시(광진교).csv', '201208_남양주시(팔당대교).csv', '201208_서울시(광진교).csv', '201209_남양주시(팔당대교).csv', '201209_서울시(광진교).csv', '201210_남양주시(팔당대교).csv', '201210_서울시(광진교).csv', '201305_남양주시(팔당대교).csv', '201305_서울시(광진교).csv', '201306_남양주시(팔당대교).csv', '201306_서울시(광진교).csv', '201307_남양주시(팔당대교).csv', '201307_서울시(광진교).csv', '201308_남양주시(팔당대교).csv', '201308_서울시(광진교).csv', '201309_남양주시(팔당대교).csv', '201309_서울시(광진교).csv', '201310_남양주시(팔당대교).csv', '201310_서울시(광진교).csv', '201405_남양주시(팔당대교).csv', '201405_서울시(광진교).csv', '201406_남양주시(팔당대교).csv', '201406_서울시(광진교).csv', '201407_남양주시(팔당대교).csv', '201407_서울시(광진교).csv', '201408_남양주시(팔당대교).csv', '201408_서울시(광진교).csv', '201409_남양주시(팔당대교).csv', '201409_서울시(광진교).csv', '201410_남양주시(팔당대교).csv', '201410_서울시(광진교).csv', '201505_남양주시(팔당대교).csv', '201505_서울시(광진교).csv', '201506_남양주시(팔당대교).csv', '201506_서울시(광진교).csv', '201507_남양주시(팔당대교).csv', '201507_서울시(광진

In [19]:
file_names = os.listdir()

# 해당 리스트에서 뽑은 파일들을 "파일명"과 "확장자"로 분리
for seoul in file_names:
#     print(seoul)
#     print((os.path.splitext(seoul)[0]).find("서울시"))
    n = (os.path.splitext(seoul)[0]).find("서울시")
    if n == 7:
        print(seoul)

201205_서울시(광진교).csv
201206_서울시(광진교).csv
201207_서울시(광진교).csv
201208_서울시(광진교).csv
201209_서울시(광진교).csv
201210_서울시(광진교).csv
201305_서울시(광진교).csv
201306_서울시(광진교).csv
201307_서울시(광진교).csv
201308_서울시(광진교).csv
201309_서울시(광진교).csv
201310_서울시(광진교).csv
201405_서울시(광진교).csv
201406_서울시(광진교).csv
201407_서울시(광진교).csv
201408_서울시(광진교).csv
201409_서울시(광진교).csv
201410_서울시(광진교).csv
201505_서울시(광진교).csv
201506_서울시(광진교).csv
201507_서울시(광진교).csv
201508_서울시(광진교).csv
201509_서울시(광진교).csv
201510_서울시(광진교).csv
201605_서울시(광진교).csv
201606_서울시(광진교).csv
201607_서울시(광진교).csv
201608_서울시(광진교).csv
201609_서울시(광진교).csv
201610_서울시(광진교).csv
201705_서울시(광진교).csv
201706_서울시(광진교).csv
201707_서울시(광진교).csv
201708_서울시(광진교).csv
201709_서울시(광진교).csv
201710_서울시(광진교).csv
201805_서울시(광진교).csv
201806_서울시(광진교).csv
201807_서울시(광진교).csv
201808_서울시(광진교).csv
201809_서울시(광진교).csv
201810_서울시(광진교).csv
201905_서울시(광진교).csv
201906_서울시(광진교).csv
201907_서울시(광진교).csv
201908_서울시(광진교).csv
201909_서울시(광진교).csv
201910_서울시(광진교).csv
202005_서울시(광진교).csv
202006_서울시(광진교).csv


In [22]:
file_names = os.listdir()

# 해당 리스트에서 뽑은 파일들을 "파일명"과 "확장자"로 분리
for nam in file_names:
#     print(nam)
#     print((os.path.splitext(nam)[0]).find("남양주시"))
    n = (os.path.splitext(nam)[0]).find("남양주시")
    if n == 7:
        print(nam)

201205_남양주시(팔당대교).csv
201206_남양주시(팔당대교).csv
201207_남양주시(팔당대교).csv
201208_남양주시(팔당대교).csv
201209_남양주시(팔당대교).csv
201210_남양주시(팔당대교).csv
201305_남양주시(팔당대교).csv
201306_남양주시(팔당대교).csv
201307_남양주시(팔당대교).csv
201308_남양주시(팔당대교).csv
201309_남양주시(팔당대교).csv
201310_남양주시(팔당대교).csv
201405_남양주시(팔당대교).csv
201406_남양주시(팔당대교).csv
201407_남양주시(팔당대교).csv
201408_남양주시(팔당대교).csv
201409_남양주시(팔당대교).csv
201410_남양주시(팔당대교).csv
201505_남양주시(팔당대교).csv
201506_남양주시(팔당대교).csv
201507_남양주시(팔당대교).csv
201508_남양주시(팔당대교).csv
201509_남양주시(팔당대교).csv
201510_남양주시(팔당대교).csv
201605_남양주시(팔당대교).csv
201606_남양주시(팔당대교).csv
201607_남양주시(팔당대교).csv
201608_남양주시(팔당대교).csv
201609_남양주시(팔당대교).csv
201610_남양주시(팔당대교).csv
201705_남양주시(팔당대교).csv
201706_남양주시(팔당대교).csv
201707_남양주시(팔당대교).csv
201708_남양주시(팔당대교).csv
201709_남양주시(팔당대교).csv
201710_남양주시(팔당대교).csv
201805_남양주시(팔당대교).csv
201806_남양주시(팔당대교).csv
201807_남양주시(팔당대교).csv
201808_남양주시(팔당대교).csv
201809_남양주시(팔당대교).csv
201810_남양주시(팔당대교).csv
201905_남양주시(팔당대교).csv
201906_남양주시(팔당대교).csv
201907_남양주시(팔당대교).csv
201908_남양주